# 11. Recommender systems

## Introduction to recommender systems

Recommendation algorithms provide the customers of companies such as Netflix, Amazon and YouTube intelligent suggestions about the items they might be interested in. There are many alternative approaches to constructing such algorithms. One of them is called **content-based filtering**, which uses the items' metadata (or descriptive characteristics) as explanatory variables to *e.g.* create models for identifying similar items than those the customer has previously expressed interest to. However, in this module we shall focus on another very common technique called **collaborative filtering**.

Collaborative filtering generates recommendations about what the user might enjoy on the basis of collected reactions from other users. This technique can be approached from two alternative directions: **item-based** (looking for items that elicit similar reactions) or **user-based** (looking for users with similar tastes) collaborative filtering. Below, we shall investigate both of these alternative approaches.  

## Item-based collaborative filtering

Collaborative filtering requires a dataset of user responses (purchase history, reviews, ratings etc.) concerning a set of items. As an example, let us consider the following simple table of user ratings given by six users (U1, ..., U6) in a scale of 1 ... 5 stars, to a set of five different movies (M1, ..., M5).


|     | U1  | U2  | U3  | U4  | U5  | U6  | 
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| M1  |  5  |     |  2  |     |  1  |  1  |
| M2  |     |  1  |  4  |  3  |  5  |  4  |
| M3  |  4  |  4  |  2  |  2  |     |  1  |
| M4  |  3  |     |  5  |     |  4  |  3  |
| M5  |  2  |  2  |     |     |  5  |  5  |